<a href="https://colab.research.google.com/github/priyanka011011/AI/blob/master/productRecommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Basic Libraries
import numpy as np
import pandas as pd

#Visualization Libraries
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px


#Text Handling Libraries
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

**DATA SET DESCRIPTION**
I have chosen Big Basket Dataset that contains various features. I have worked on a product recommendation system considering the various features into account.

Importing the Dataset

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/BigBasketProducts.csv",index_col='index')  
df.head()                                                

,product,category,sub_category,brand,sale_price,market_price,type,rating,description
index,,,,,,,,,
1,Garlic Oil - Vegetarian Capsule 500 mg,Beauty & Hygiene,Hair Care,Sri Sri Ayurveda,220.0,220.0,Hair Oil & Serum,4.1,This Product contains Garlic Oil that is known...
2,Water Bottle - Orange,"Kitchen, Garden & Pets",Storage & Accessories,Mastercook,180.0,180.0,Water & Fridge Bottles,2.3,"Each product is microwave safe (without lid), ..."
3,"Brass Angle Deep - Plain, No.2",Cleaning & Household,Pooja Needs,Trm,119.0,250.0,Lamp & Lamp Oil,3.4,"A perfect gift for all occasions, be it your m..."
4,Cereal Flip Lid Container/Storage Jar - Assort...,Cleaning & Household,Bins & Bathroom Ware,Nakoda,149.0,176.0,"Laundry, Storage Baskets",3.7,Multipurpose container with an attractive desi...
5,Creme Soft Soap - For Hands & Body,Beauty & Hygiene,Bath & Hand Wash,Nivea,162.0,162.0,Bathing Bars & Soaps,4.4,Nivea Creme Soft Soap gives your skin the best...


In [ ]:
df.shape 

(27555, 9)

In [ ]:
df.head() 

,product,category,sub_category,brand,sale_price,market_price,type,rating,description
index,,,,,,,,,
1,Garlic Oil - Vegetarian Capsule 500 mg,Beauty & Hygiene,Hair Care,Sri Sri Ayurveda,220.0,220.0,Hair Oil & Serum,4.1,This Product contains Garlic Oil that is known...
2,Water Bottle - Orange,"Kitchen, Garden & Pets",Storage & Accessories,Mastercook,180.0,180.0,Water & Fridge Bottles,2.3,"Each product is microwave safe (without lid), ..."
3,"Brass Angle Deep - Plain, No.2",Cleaning & Household,Pooja Needs,Trm,119.0,250.0,Lamp & Lamp Oil,3.4,"A perfect gift for all occasions, be it your m..."
4,Cereal Flip Lid Container/Storage Jar - Assort...,Cleaning & Household,Bins & Bathroom Ware,Nakoda,149.0,176.0,"Laundry, Storage Baskets",3.7,Multipurpose container with an attractive desi...
5,Creme Soft Soap - For Hands & Body,Beauty & Hygiene,Bath & Hand Wash,Nivea,162.0,162.0,Bathing Bars & Soaps,4.4,Nivea Creme Soft Soap gives your skin the best...


Using null() function, I am checking the presence of null values in each feature column and sum() is counting the total number of null values.

In [ ]:
df.isnull().sum() 

product            1
category           0
sub_category       0
brand              1
sale_price         0
market_price       0
type               0
rating          8626
description      115
dtype: int64

To get a clear insight of null data, I have converted the null data into percentage.

In [ ]:
print('Percentage Null Data In Each Column')
print('-'*50)
for col in df.columns:
    null_count = df[col].isnull().sum()
    total_count = df.shape[0]
    print("{} : {:.2f}".format(col,null_count/total_count * 100)) 

Percentage Null Data In Each Column
--------------------------------------------------
product : 0.00
category : 0.00
sub_category : 0.00
brand : 0.00
sale_price : 0.00
market_price : 0.00
type : 0.00
rating : 31.30
description : 0.42


In [ ]:
print('Total Null Data')
null_count = df.isnull().sum().sum()
total_count = np.product(df.shape)
print("{:.2f}".format(null_count/total_count *100)) 

Total Null Data
3.53


We observe that 3% of the data is null and 31% of the rating is missing. So, we drop the null values and work with the remaining 69% data as this much data will be enough for recommendation purposes. 

In [ ]:
df = df.dropna()

dropna() function removes the rows having null values. 

In [ ]:
df.isnull().sum()

product         0
category        0
sub_category    0
brand           0
sale_price      0
market_price    0
type            0
rating          0
description     0
dtype: int64

In [ ]:
df.shape

(18840, 9)

Now, we can form the recommendation system using 18000+ products which is enough.

In [ ]:
df.head()

,product,category,sub_category,brand,sale_price,market_price,type,rating,description
index,,,,,,,,,
1,Garlic Oil - Vegetarian Capsule 500 mg,Beauty & Hygiene,Hair Care,Sri Sri Ayurveda,220.0,220.0,Hair Oil & Serum,4.1,This Product contains Garlic Oil that is known...
2,Water Bottle - Orange,"Kitchen, Garden & Pets",Storage & Accessories,Mastercook,180.0,180.0,Water & Fridge Bottles,2.3,"Each product is microwave safe (without lid), ..."
3,"Brass Angle Deep - Plain, No.2",Cleaning & Household,Pooja Needs,Trm,119.0,250.0,Lamp & Lamp Oil,3.4,"A perfect gift for all occasions, be it your m..."
4,Cereal Flip Lid Container/Storage Jar - Assort...,Cleaning & Household,Bins & Bathroom Ware,Nakoda,149.0,176.0,"Laundry, Storage Baskets",3.7,Multipurpose container with an attractive desi...
5,Creme Soft Soap - For Hands & Body,Beauty & Hygiene,Bath & Hand Wash,Nivea,162.0,162.0,Bathing Bars & Soaps,4.4,Nivea Creme Soft Soap gives your skin the best...


using value_count() function, we are counting the total elements present in each column so that we can use the data for plotting.

In [ ]:
counts = df['category'].value_counts()
counts_df = pd.DataFrame({'category': counts.index, 'counts': counts.values})
counts_df  

,category,counts
0,Beauty & Hygiene,5460
1,"Kitchen, Garden & Pets",2494
2,Snacks & Branded Foods,2468
3,Gourmet & World Food,2364
4,"Foodgrains, Oil & Masala",2173
5,Cleaning & Household,2091
6,"Bakery, Cakes & Dairy",665
7,Beverages,630
8,Baby Care,495


In [ ]:
px.bar(data_frame=counts_df,
 x='category',
 y='counts',
 color='counts',
 color_continuous_scale='reds',
 text_auto=True,
 title=f'Count of Items in Each Category') 

Its clear from the graph that category "Beauty & Hygiene" is having the highest number of product sales.

In [ ]:
counts = df['sub_category'].value_counts()

counts_df_1 = pd.DataFrame({'Category':counts.index,'Counts':counts.values})[:10]
counts_df_1


,Category,Counts
0,Skin Care,1641
1,Hair Care,818
2,Bath & Hand Wash,808
3,Masalas & Spices,764
4,Storage & Accessories,658
5,Men's Grooming,649
6,Fragrances & Deos,627
7,Crockery & Cutlery,621
8,Ready To Cook & Eat,557
9,Organic Staples,550


In [ ]:
px.bar(data_frame=counts_df_1,
 x='Category',
 y='Counts',
 color='Counts',
 color_continuous_scale='greens',
 text_auto=True,
 title=f'Top 10 Bought Sub_Categories')

In [ ]:
counts = df['brand'].value_counts()

counts_df_brand = pd.DataFrame({'Brand Name':counts.index,'Counts':counts.values})[:10]
counts_df_brand

,Brand Name,Counts
0,bb Royal,278
1,BB Home,172
2,Amul,153
3,Himalaya,139
4,Cello,104
5,BIOTIQUE,103
6,DP,101
7,Keya,101
8,Organic Tattva,99
9,MTR,97


In [ ]:
px.bar(data_frame=counts_df_brand,
 x='Brand Name',
 y='Counts',
 color='Counts',
 color_continuous_scale='blues',
 text_auto=True,
 title=f'Top 10 Brand Items based on Item Counts')

By plotting the bar graph, the top 10 brands of the items are clearly visible.

In [ ]:
counts = df['type'].value_counts()

counts_df_type = pd.DataFrame({'Type':counts.index,'Counts':counts.values})[:10]
counts_df_type

,Type,Counts
0,Face Care,1094
1,Men's Deodorants,404
2,Shampoo & Conditioner,390
3,Blended Masalas,343
4,Containers Sets,332
5,Bathing Bars & Soaps,322
6,Glassware,263
7,Body Care,251
8,Namkeen & Savoury Snacks,234
9,Hand Wash & Sanitizers,212


In [ ]:
px.bar(data_frame=counts_df_type,
 x='Type',
 y='Counts',
 color='Counts',
 color_continuous_scale='blues',
 text_auto=True,
 title=f'Top 10 Types of Products based on Item Counts')

Next, I have plotted the Type of the items to see which items were solds the most. And It is clear that Face Care items were sold the most followed by Men's Deodarants.

                   #DEMOGRAPHIC FILTER RECOMMENDOR

> It is like recommending items based on a feature. Say, the top ten items in a category are recommended first to the customers.





In [ ]:
def sort_recommendor(col='rating',sort_type = False):
    """
    A recommendor based on sorting products on the column passed.
    Arguments to be passed.
    
    col: The Feature to be used for recommendation.
    sort_type: True for Ascending Order
    """
    rated_recommend = df.copy()
    if rated_recommend[col].dtype == 'O':
        col='rating'
    rated_recommend = rated_recommend.sort_values(by=col,ascending = sort_type)
    return rated_recommend[['product','brand','sale_price','rating']].head(10)

In [ ]:
help(sort_recommendor)

Help on function sort_recommendor in module __main__:

sort_recommendor(col='rating', sort_type=False)
    A recommendor based on sorting products on the column passed.
    Arguments to be passed.
    
    col: The Feature to be used for recommendation.
    sort_type: True for Ascending Order



In [ ]:
sort_recommendor(col='sale_price',sort_type=True)

,product,brand,sale_price,rating
index,,,,
21313,Serum,Livon,3.0,2.5
18291,Sugar Coated Chocolate,Cadbury Gems,5.0,4.2
21229,Dish Shine Bar,Exo,5.0,4.2
14539,Cadbury Perk - Chocolate Bar,Cadbury,5.0,4.2
19539,Layer Cake - Chocolate,Winkies,5.0,4.2
2979,Sugar Free Chewing Gum - Mixed Fruit,Orbit,5.0,4.2
15927,Dreams Cup Cake - Choco,Elite,5.0,3.9
6015,Good Day Butter Cookies,Britannia,5.0,4.1
27414,Layer Cake - Orange,Winkies,5.0,4.1


We can see that the top product has a rating 2.5 which is bad. So it is better to filter the data more by rating. 

In [ ]:
mean_rating= df['rating'].mean()
mean_rating                       

3.9430626326963902

So the average rating of products is 3.94 Let's use 3.5 as the threshold.

In [ ]:
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['description'])
tfidf_matrix.shape

(18840, 23342)

Now, we use linear_kernel to evaluate our similarity score.

In [ ]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_sim

array([[1.        , 0.01632718, 0.00999603, ..., 0.01056047, 0.01133156,
        0.        ],
       [0.01632718, 1.        , 0.00719713, ..., 0.        , 0.        ,
        0.        ],
       [0.00999603, 0.00719713, 1.        , ..., 0.00635776, 0.        ,
        0.        ],
       ...,
       [0.01056047, 0.        , 0.00635776, ..., 1.        , 0.        ,
        0.        ],
       [0.01133156, 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [ ]:
indices = pd.Series(df.index, index=df['product']).drop_duplicates()

def get_recommendations_1(title, cosine_sim=cosine_sim):
    
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    movie_indices = [i[0] for i in sim_scores]
    return df['product'].iloc[movie_indices]

In [ ]:
get_recommendations_1('Water Bottle - Orange')


index
1677         Brass Nanda Stand Goblets - No.1
2162         Brass Kachua Stand Deepam - No.1
2756     Brass Angle Deep Stand - Plain, No.2
5400       Brass Lakshmi Deepam - Plain, No.2
6520                Brass Kuber Deepam - No.1
10504               Brass Kuber Deepam - No.2
11226    Brass Angle Deep Stand - Plain, No.3
11504    Brass Angle Deep Stand - Plain, No.1
12699        Brass Kachua Stand Deepam - No.2
18572               Brass Kuber Deepam - No.3
Name: product, dtype: object

In [ ]:
get_recommendations_1('Cadbury Perk - Chocolate Bar')

index
27049                        Pickle - Mixed
6601                  Pickle - Kaduku Mango
17934                Pickle - Mix Vegetable
27105                        Pickle - Prawn
3962                  Pickle - Tender Mango
16875             Olive Oil - Carrot Pickle
3444                     Pickle - Cut Mango
17237      Andhra Special Red Chilli Pickle
27234    Pickle - Lime (South Indian Style)
4955                    Pickle - Gooseberry
Name: product, dtype: object

Our search was chocolate yet we got pickle and prawn recommended.
We need to optimize this based on category, sub_category and brand.

In [ ]:
df2 = df.copy()

In [ ]:
df2.head()

,product,category,sub_category,brand,sale_price,market_price,type,rating,description
index,,,,,,,,,
1,Garlic Oil - Vegetarian Capsule 500 mg,Beauty & Hygiene,Hair Care,Sri Sri Ayurveda,220.0,220.0,Hair Oil & Serum,4.1,This Product contains Garlic Oil that is known...
2,Water Bottle - Orange,"Kitchen, Garden & Pets",Storage & Accessories,Mastercook,180.0,180.0,Water & Fridge Bottles,2.3,"Each product is microwave safe (without lid), ..."
3,"Brass Angle Deep - Plain, No.2",Cleaning & Household,Pooja Needs,Trm,119.0,250.0,Lamp & Lamp Oil,3.4,"A perfect gift for all occasions, be it your m..."
4,Cereal Flip Lid Container/Storage Jar - Assort...,Cleaning & Household,Bins & Bathroom Ware,Nakoda,149.0,176.0,"Laundry, Storage Baskets",3.7,Multipurpose container with an attractive desi...
5,Creme Soft Soap - For Hands & Body,Beauty & Hygiene,Bath & Hand Wash,Nivea,162.0,162.0,Bathing Bars & Soaps,4.4,Nivea Creme Soft Soap gives your skin the best...


In [ ]:
df2.shape

(18840, 9)

Notice that a product can be in multiple catergories and sub_categories and they are separated with a &.
Let's split them into a list for futher processes.

In [ ]:
rmv_spc = lambda a:a.strip()
get_list = lambda a:list(map(rmv_spc,re.split('& |, |\*|\n', a))) 

In [ ]:
get_list('A & B, C')

['A', 'B', 'C']

In [ ]:
for col in ['category', 'sub_category', 'type']:
    df2[col] = df2[col].apply(get_list) 

In [ ]:
df2.head()

,product,category,sub_category,brand,sale_price,market_price,type,rating,description
index,,,,,,,,,
1,Garlic Oil - Vegetarian Capsule 500 mg,"[Beauty, Hygiene]",[Hair Care],Sri Sri Ayurveda,220.0,220.0,"[Hair Oil, Serum]",4.1,This Product contains Garlic Oil that is known...
2,Water Bottle - Orange,"[Kitchen, Garden, Pets]","[Storage, Accessories]",Mastercook,180.0,180.0,"[Water, Fridge Bottles]",2.3,"Each product is microwave safe (without lid), ..."
3,"Brass Angle Deep - Plain, No.2","[Cleaning, Household]",[Pooja Needs],Trm,119.0,250.0,"[Lamp, Lamp Oil]",3.4,"A perfect gift for all occasions, be it your m..."
4,Cereal Flip Lid Container/Storage Jar - Assort...,"[Cleaning, Household]","[Bins, Bathroom Ware]",Nakoda,149.0,176.0,"[Laundry, Storage Baskets]",3.7,Multipurpose container with an attractive desi...
5,Creme Soft Soap - For Hands & Body,"[Beauty, Hygiene]","[Bath, Hand Wash]",Nivea,162.0,162.0,"[Bathing Bars, Soaps]",4.4,Nivea Creme Soft Soap gives your skin the best...


There is duplicacy between lowercase and uppercase words so all the words are converted into lowercase. The reason because our recommendation system should not consider '**Chocolate**' of **'Chocolate IceCream' **and '**Chocolate Bar**' as the same.

In [ ]:
def cleaner(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''

In [ ]:
for col in ['category', 'sub_category', 'type','brand']:
    df2[col] = df2[col].apply(cleaner)

In [ ]:
df2.head()

,product,category,sub_category,brand,sale_price,market_price,type,rating,description
index,,,,,,,,,
1,Garlic Oil - Vegetarian Capsule 500 mg,"[beauty, hygiene]",[haircare],srisriayurveda,220.0,220.0,"[hairoil, serum]",4.1,This Product contains Garlic Oil that is known...
2,Water Bottle - Orange,"[kitchen, garden, pets]","[storage, accessories]",mastercook,180.0,180.0,"[water, fridgebottles]",2.3,"Each product is microwave safe (without lid), ..."
3,"Brass Angle Deep - Plain, No.2","[cleaning, household]",[poojaneeds],trm,119.0,250.0,"[lamp, lampoil]",3.4,"A perfect gift for all occasions, be it your m..."
4,Cereal Flip Lid Container/Storage Jar - Assort...,"[cleaning, household]","[bins, bathroomware]",nakoda,149.0,176.0,"[laundry, storagebaskets]",3.7,Multipurpose container with an attractive desi...
5,Creme Soft Soap - For Hands & Body,"[beauty, hygiene]","[bath, handwash]",nivea,162.0,162.0,"[bathingbars, soaps]",4.4,Nivea Creme Soft Soap gives your skin the best...


In [ ]:
def couple(x):
    return ' '.join(x['category']) + ' ' + ' '.join(x['sub_category']) + ' '+x['brand']+' ' +' '.join( x['type'])
df2['soup'] = df2.apply(couple, axis=1)

In [ ]:
df2['soup'].head()


index
1    beauty hygiene haircare srisriayurveda hairoil...
2    kitchen garden pets storage accessories master...
3       cleaning household poojaneeds trm lamp lampoil
4    cleaning household bins bathroomware nakoda la...
5    beauty hygiene bath handwash nivea bathingbars...
Name: soup, dtype: object

We need to count the String Vectors and then Cosine Similarity Score.

In [ ]:
df2.head()

,product,category,sub_category,brand,sale_price,market_price,type,rating,description,soup
index,,,,,,,,,,
1,Garlic Oil - Vegetarian Capsule 500 mg,"[beauty, hygiene]",[haircare],srisriayurveda,220.0,220.0,"[hairoil, serum]",4.1,This Product contains Garlic Oil that is known...,beauty hygiene haircare srisriayurveda hairoil...
2,Water Bottle - Orange,"[kitchen, garden, pets]","[storage, accessories]",mastercook,180.0,180.0,"[water, fridgebottles]",2.3,"Each product is microwave safe (without lid), ...",kitchen garden pets storage accessories master...
3,"Brass Angle Deep - Plain, No.2","[cleaning, household]",[poojaneeds],trm,119.0,250.0,"[lamp, lampoil]",3.4,"A perfect gift for all occasions, be it your m...",cleaning household poojaneeds trm lamp lampoil
4,Cereal Flip Lid Container/Storage Jar - Assort...,"[cleaning, household]","[bins, bathroomware]",nakoda,149.0,176.0,"[laundry, storagebaskets]",3.7,Multipurpose container with an attractive desi...,cleaning household bins bathroomware nakoda la...
5,Creme Soft Soap - For Hands & Body,"[beauty, hygiene]","[bath, handwash]",nivea,162.0,162.0,"[bathingbars, soaps]",4.4,Nivea Creme Soft Soap gives your skin the best...,beauty hygiene bath handwash nivea bathingbars...


In [ ]:
df2.to_csv('data_cleaned_1.csv')

In [ ]:
count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(df2['soup'])

In [ ]:
cosine_sim2 = cosine_similarity(count_matrix, count_matrix)
cosine_sim2

array([[1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.27216553],
       [0.        , 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.27216553, 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [ ]:
df2 = df2.reset_index()
indices = pd.Series(df2.index, index=df2['product'])

In [ ]:
def get_recommendations_2(title, cosine_sim=cosine_sim):
    idx = indices[title]

    sim_scores = list(enumerate(cosine_sim[idx]))

    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    sim_scores = sim_scores[1:11]

    movie_indices = [i[0] for i in sim_scores]

    return df2['product'].iloc[movie_indices]


Comparing Old and New Recommendations

In [ ]:
old_rec = get_recommendations_1('Peri-Peri Sweet Potato Chips').values
new_rec = get_recommendations_2('Peri-Peri Sweet Potato Chips', cosine_sim2).values

pd.DataFrame({'Old Recommendor': old_rec,'New Recommendor':new_rec})

,Old Recommendor,New Recommendor
0,Salsa Sweet Potato Chips,High Protein Soya Chips
1,South African Style Peri Peri Flavour,Chia Seeds Chips
2,Peri Peri Crackers - 100% Wholewheat,Peri-Peri Sweet Potato Chips
3,Banana Chips Masala,Sour Cream & Onion
4,Mad Angles - Very Peri Peri,"Nacho Chips - Cheese With Herbs, No Onion, No ..."
5,Baked Masala - Chips Methi,Nacho Crisps - Cheese & Herbs
6,Nachoz - Sizzling Peri Peri,Shells - Taco
7,Nachoz - Sizzling Peri Peri,6 Corn Wraps Try It With Prawns & Avocado
8,Tapioca Chips - Classic Salt,On The Go - Peri Peri Nachos & Salsa Dip
9,African Peri Peri Seasoning,Chips - Keralas Nendran Banana


In [ ]:
old_rec = get_recommendations_1('Shells - Taco').values
new_rec = get_recommendations_2('Shells - Taco', cosine_sim2).values 

pd.DataFrame({'Old Recommendor': old_rec,'New Recommendor':new_rec})      

,Old Recommendor,New Recommendor
0,Mexican Seasoning,Nacho Chips - Peri Peri
1,"Snack Time - Tacos Nachos Masala, Mexican Flavour",Nacho Round Chips
2,Taco Shells,Nachos Peri Peri
3,Taco Shells - 6 Inches,Nachos Tangy Tomato
4,Taco Shells - 4 Inches,Tortilla - Original Wrap
5,Go Natural Cheese - Shredded Mexican Style,Tortilla Chips - Jalapeno
6,Breakfast Mix - Upma,Nacho Chips - Crunchy Pizza
7,Tortilla Chips - Chilli,Tortilla Chips - Chilli
8,Chilli Flakes - Red,Tortilla Chips - Cheese
9,Nachoz - Sizzling Peri Peri,Sour Cream & Onion


New Recommendations are more preferrable.